<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
# !pip install --upgrade seaborn==0.10.1
# !pip install --upgrade scikit-learn==0.23.1
# !pip install --upgrade matplotlib==3.2.0
# !pip install --upgrade pandas==1.0.4
# !pip install --upgrade qiskit==0.19.6 
# !pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [2]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [5]:
#data_path='./dataset/'
data_path='../'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

#ind = np.random.permutation(data.shape[0])

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [6]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)
np.random.shuffle(four_datapoints)
np.random.shuffle(nine_datapoints)

train_num = 250
test_num = 100

training_input = {4:four_datapoints[:train_num], 9:nine_datapoints[:train_num]}
test_input = {4:four_datapoints[-test_num:], 9:nine_datapoints[-test_num:]}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [7]:
%%write_and_run feature_map.py

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def feature_map(): 

    ckt9_depth=3
    num_qubits=6
    x = ParameterVector('x', length=3)
    feature_map=QuantumCircuit(num_qubits)
    
    for i in range(ckt9_depth):
        for j in range(num_qubits):
            feature_map.h(j)
        for j in range(2):
            feature_map.crz((np.pi-x[j])*(np.pi-x[j+1]),j,j+1)
        for j in range(2):
            feature_map.crz((np.pi-x[j]*x[j])*(np.pi-x[j+1]*x[j+1]),j+3,j+4)
        for j in range(3):
            feature_map.rx(x[j],j)
        for j in range(3):
            feature_map.rx(x[j]*x[j],j+3)

    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

In [8]:
feature_map().draw()

┌───┐                                                                 »
q_0: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤   ┌────────────────────────────┴────────────────────────────┐   »
q_1: ┤ H ├───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
     ├───┤   └─────────────────────────────────────────────────────────┘   »
q_2: ┤ H ├─────────────────────────────────────────────────────────────────»
     ├───┤                                                                 »
q_3: ┤ H ├────────────────────────────────■────────────────────────────────»
     ├───┤┌───────────────────────────────┴───────────────────────────────┐»
q_4: ┤ H ├┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
     ├───┤└───────────────────────────────────────────────────────────────┘»
q_5: ┤ H ├─────────────────────────────────────────────────────────────────»
     └───┘                                                                 »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_5: ┤ RZ((3.14159265358979 - x[1]**2)*(3.14159265358979 - x[2]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«          ┌───┐          »
«q_0: ─────┤ H ├──────────»
«       ┌──┴───┴───┐ ┌───┐»
«q_1: ──┤ RX(x[1]) ├─┤ H ├»
«       ├──────────┤ ├───┤»
«q_2: ──┤ RX(x[2]) ├─┤ H ├»
«       └──┬───┬───┘ └───┘»
«q_3: ─────┤ H ├──────────»
«     ┌────┴───┴────┐┌───┐»
«q_4: ┤ RX(x[1]**2) ├┤ H ├»
«     ├─────────────┤├───┤»
«q_5: ┤ RX(x[2]**2) ├┤ H ├»
«     └─────────────┘└───┘»
«                                                                      »
«q_0: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_1: ───┤ RZ((3.14159265358979 - x[0])*(3.14159265358979 - x[1])) ├───»
«        └─────────────────────────────────────────────────────────┘   »
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_3: ────────────────────────────────■────────────────────────────────»
«     ┌───────────────────────────────┴───────────────────────────────┐»
«q_4: ┤ RZ((3.14159265358979 - x[0]**2)*(3.14159265358979 - x[1]**2)) ├»
«     └───────────────────────────────────────────────────────────────┘»
«q_5: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                                ┌──────────┐                          »
«q_0: ───────────────────────────┤ RX(x[0]) ├──────────────────────────»
«                                └──────────┘                          »
«q_1: ────────────────────────────────■────────────────────────────────»
«        ┌────────────────────────────┴────────────────────────────┐   »
«q_2: ───┤ RZ((3.14159265358979 - x[1])*(3.14159265358979 - x[2])) ├───»
«        └─────────────────────┬─────────────┬─────────────────────┘   »
«q_3: ─────────────────────────┤ RX(x[0]**2) ├─────────────────────────»
«                              └─────────────┘                         »
«q_4: ────────────────────────────────■────────────────────────────────»
«     ┌─────

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [9]:
%%write_and_run variational_circuit.py

def variational_circuit():

    ckt5_depth=2
    num_qubits=6
    y = ParameterVector('y', length=num_qubits*(num_qubits+3)*ckt5_depth)
    var_circuit=QuantumCircuit(num_qubits)
    ii = 0
    
    for i in range(ckt5_depth):
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            for k in range(num_qubits):
                if j != k:
                    var_circuit.crz(y[ii], num_qubits-1-j, num_qubits-1-k)
                    ii += 1
        for j in range(num_qubits):
            var_circuit.rx(y[ii], j)
            ii += 1
        for j in range(num_qubits):
            var_circuit.rz(y[ii], j)
            ii += 1

    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

In [10]:
variational_circuit().draw()

┌──────────┐ ┌──────────┐                                       »
q_0: ┤ RX(y[0]) ├─┤ RZ(y[6]) ├───────────────────────────────────────»
     ├──────────┤ ├──────────┤                                       »
q_1: ┤ RX(y[1]) ├─┤ RZ(y[7]) ├───────────────────────────────────────»
     ├──────────┤ ├──────────┤                          ┌───────────┐»
q_2: ┤ RX(y[2]) ├─┤ RZ(y[8]) ├──────────────────────────┤ RZ(y[14]) ├»
     ├──────────┤ ├──────────┤             ┌───────────┐└─────┬─────┘»
q_3: ┤ RX(y[3]) ├─┤ RZ(y[9]) ├─────────────┤ RZ(y[13]) ├──────┼──────»
     ├──────────┤┌┴──────────┤┌───────────┐└─────┬─────┘      │      »
q_4: ┤ RX(y[4]) ├┤ RZ(y[10]) ├┤ RZ(y[12]) ├──────┼────────────┼──────»
     ├──────────┤├───────────┤└─────┬─────┘      │            │      »
q_5: ┤ RX(y[5]) ├┤ RZ(y[11]) ├──────■────────────■────────────■──────»
     └──────────┘└───────────┘                                       »
«                  ┌───────────┐                                       »
«q_0: ─────────────┤ RZ(y[16]) ├───────────────────────────────────────»
«     ┌───────────┐└─────┬─────┘                                       »
«q_1: ┤ RZ(y[15]) ├──────┼─────────────────────────────────────────────»
«     └─────┬─────┘      │                                ┌───────────┐»
«q_2: ──────┼────────────┼────────────────────────────────┤ RZ(y[19]) ├»
«           │            │                   ┌───────────┐└─────┬─────┘»
«q_3: ──────┼────────────┼───────────────────┤ RZ(y[18]) ├──────┼──────»
«           │            │                   └─────┬─────┘      │      »
«q_4: ──────┼────────────┼────────────■────────────■────────────■──────»
«           │            │      ┌─────┴─────┐                          »
«q_5: ──────■────────────■──────┤ RZ(y[17]) ├──────────────────────────»
«                               └───────────┘                          »
«                               ┌───────────┐                          »
«q_0: ──────────────────────────┤ RZ(y[21]) ├──────────────────────────»
«                  ┌───────────┐└─────┬─────┘                          »
«q_1: ─────────────┤ RZ(y[20]) ├──────┼────────────────────────────────»
«                  └─────┬─────┘      │                   ┌───────────┐»
«q_2: ───────────────────┼────────────┼───────────────────┤ RZ(y[24]) ├»
«                        │            │                   └─────┬─────┘»
«q_3: ──────■────────────┼────────────┼────────────■────────────■──────»
«           │            │            │      ┌─────┴─────┐             »
«q_4: ──────┼────────────■────────────■──────┤ RZ(y[23]) ├─────────────»
«     ┌─────┴─────┐                          └───────────┘             »
«q_5: ┤ RZ(y[22]) ├────────────────────────────────────────────────────»
«     └───────────┘                                                    »
«                               ┌───────────┐                          »
«q_0: ──────────────────────────┤ RZ(y[26]) ├──────────────────────────»
«     ┌───────────┐             └─────┬─────┘                          »
«q_1: ┤ RZ(y[25]) ├───────────────────┼────────────────────────────────»
«     └─────┬─────┘                   │                                »
«q_2: ──────┼────────────■────────────┼────────────■────────────■──────»
«           │            │            │            │      ┌─────┴─────┐»
«q_3: ──────■────────────┼────────────■────────────┼──────┤ RZ(y[29]) ├»
«                        │                   ┌─────┴─────┐└───────────┘»
«q_4: ───────────────────┼───────────────────┤ RZ(y[28]) ├─────────────»
«                  ┌─────┴─────┐             └───────────┘             »
«q_5: ─────────────┤ RZ(y[27]) ├───────────────────────────────────────»
«                  └───────────┘                                       »
«                  ┌───────────┐                                       »
«q_0: ─────────────┤ RZ(y[31]) ├───────────────────────────────────────»
«     ┌───────────┐└─────┬─────┘                                       »
«q_1: ┤ RZ(y[30]

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [13]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    from qiskit.aqua.components.optimizers import SPSA
    
    # create an optimizer object
    cls_opt = SPSA(max_trials=500)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [14]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [15]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 1059

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)
#quantum_instance = QuantumInstance(backend, shots=1024, backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [16]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [ 0.18341821  1.87048989  1.3160707   0.04310941  0.50793032 -0.37410945
  1.6025248  -0.69813195  1.39753064  0.86877685  1.52163701 -0.97547846
 -0.96896903  2.16265001 -0.80190844  1.22208461  0.60705435  0.38328284
  0.00731544  1.47332097 -0.09689338 -0.45368268  0.0297584  -0.03537005
  0.53398471  0.17838125 -1.55990155  0.31004605  0.70758076  0.23663798
  1.41500345  0.52430888 -0.61059212 -0.6321073   0.7465184   0.06221418
 -0.03014402 -0.40222153  1.41059371 -1.61036808  1.85142079 -2.2166023
  0.05906622  0.64165209 -0.58846996 -1.82656003  1.74598481  1.17845613
 -0.33556587  0.86488404  0.32935361  0.64286605 -0.11686443  0.15870358
 -0.29020213  0.91905794  0.64252949 -1.65695396  0.302944   -1.80048367
 -0.43264157  1.50919106  1.27364095 -2.58896761  0.07813819  0.66957323
 -1.23094613  0.84754609  1.2487993   0.05085518  0.53951039  1.69711942
 -0.75846842  0.36719942  0.32936845  0.33845686 -1.03493722  1.033103
  0.20465951  0.63535164  0.

eval_count: 6
var_params: [ 0.18341821  1.67048989  1.3160707  -0.15689059  0.50793032 -0.17410945
  1.6025248  -0.69813195  1.39753064  0.66877685  1.52163701 -1.17547846
 -0.96896903  2.16265001 -0.60190844  1.22208461  0.40705435  0.58328284
  0.20731544  1.47332097 -0.09689338 -0.25368268 -0.1702416   0.16462995
  0.73398471  0.17838125 -1.55990155  0.31004605  0.90758076  0.23663798
  1.41500345  0.52430888 -0.61059212 -0.6321073   0.7465184   0.06221418
 -0.23014402 -0.60222153  1.61059371 -1.61036808  1.85142079 -2.4166023
  0.05906622  0.64165209 -0.38846996 -1.62656003  1.54598481  0.97845613
 -0.33556587  0.66488404  0.32935361  0.44286605  0.08313557  0.15870358
 -0.09020213  0.91905794  0.64252949 -1.65695396  0.102944   -1.80048367
 -0.43264157  1.50919106  1.27364095 -2.38896761  0.07813819  0.66957323
 -1.43094613  1.04754609  1.4487993   0.05085518  0.73951039  1.49711942
 -0.55846842  0.36719942  0.32936845  0.33845686 -1.03493722  1.033103
  0.40465951  0.83535164  0.

eval_count: 12
var_params: [-0.01658179  1.87048989  1.1160707   0.04310941  0.50793032 -0.37410945
  1.8025248  -0.49813195  1.19753064  0.86877685  1.32163701 -0.97547846
 -0.76896903  1.96265001 -0.80190844  1.02208461  0.60705435  0.58328284
  0.00731544  1.47332097  0.10310662 -0.25368268 -0.1702416  -0.03537005
  0.73398471  0.17838125 -1.75990155  0.11004605  0.90758076  0.43663798
  1.61500345  0.32430888 -0.41059212 -0.8321073   0.7465184  -0.13778582
 -0.23014402 -0.40222153  1.41059371 -1.61036808  1.65142079 -2.2166023
  0.25906622  0.84165209 -0.38846996 -1.62656003  1.54598481  0.97845613
 -0.53556587  0.66488404  0.12935361  0.44286605 -0.11686443 -0.04129642
 -0.09020213  0.91905794  0.64252949 -1.65695396  0.102944   -1.80048367
 -0.43264157  1.30919106  1.27364095 -2.58896761  0.07813819  0.46957323
 -1.23094613  1.04754609  1.4487993   0.25085518  0.73951039  1.69711942
 -0.75846842  0.36719942  0.12936845  0.33845686 -1.23493722  1.233103
  0.20465951  0.83535164  0

eval_count: 18
var_params: [ 0.18341821  1.87048989  1.3160707   0.04310941  0.30793032 -0.17410945
  1.6025248  -0.69813195  1.19753064  0.86877685  1.32163701 -1.17547846
 -0.96896903  2.16265001 -0.80190844  1.22208461  0.60705435  0.38328284
  0.00731544  1.47332097 -0.09689338 -0.45368268 -0.1702416   0.16462995
  0.73398471 -0.02161875 -1.55990155  0.11004605  0.70758076  0.43663798
  1.61500345  0.32430888 -0.61059212 -0.8321073   0.5465184  -0.13778582
 -0.23014402 -0.40222153  1.41059371 -1.41036808  1.65142079 -2.4166023
  0.25906622  0.84165209 -0.58846996 -1.62656003  1.74598481  1.17845613
 -0.53556587  0.86488404  0.32935361  0.44286605 -0.11686443  0.15870358
 -0.29020213  0.71905794  0.44252949 -1.65695396  0.302944   -2.00048367
 -0.63264157  1.30919106  1.27364095 -2.38896761  0.07813819  0.66957323
 -1.23094613  1.04754609  1.2487993   0.25085518  0.73951039  1.69711942
 -0.75846842  0.36719942  0.32936845  0.53845686 -1.23493722  1.033103
  0.40465951  0.83535164  1

eval_count: 24
var_params: [-0.85369545  2.70760355  1.0225187  -0.99400424  0.60148231  0.66300421
  2.63963846  0.3389817   2.2346443   0.75421979  2.17975573 -0.88192646
 -1.06252103  2.04809294 -0.50835644  0.3639659   1.44416801 -0.27483588
  0.09275838  1.58787803 -0.75501209 -0.36823974 -0.05568454  0.82274867
  0.61942765  0.06382419 -0.7227879   0.40359805  1.00113276  0.32208092
  0.75688474  0.40975182  0.42652153 -0.5385553   0.4529664  -0.23133782
  0.80696964 -1.26034024  1.31704171 -1.70392008  0.81430714 -2.5101543
 -0.77804743  0.75620915 -1.24658868 -1.53300803  1.83953681  1.06389907
 -1.37267952  0.00676533  0.24391067 -0.21525267 -0.77498315  0.25225558
  0.74691153  0.06093922  1.30064821 -1.95050596  1.14005765 -1.91504073
 -1.46975522  0.4720774   1.18008895 -3.24708633  0.73625691  0.76312523
 -1.13739413  0.93298903  0.59068059  1.08796883  0.44595839  0.83900071
 -0.64391136  1.02531813  0.42292045  1.37557052 -1.14949428  0.939551
  0.49821151 -0.02276707  1

...

eval_count: 95
var_params: [-0.25796128  2.81826369  1.97331603 -1.08176756  0.48317736 -0.1869739
  2.21186711 -0.29722775  1.14086566  1.38173778  2.09479639 -2.46358856
  0.34851179  1.03765985 -1.6439604   0.27964731  1.19920277 -0.20809671
 -0.55077766  1.84220835 -1.46483865 -0.50416158 -0.16268739  0.93049567
 -0.87164079 -0.64323686 -1.32788306 -1.60326239  0.29677321  0.11159667
 -0.43853275  0.68223152  1.32262923 -0.20678413  0.31839448 -0.07654059
 -0.23878689 -1.99868969  1.37828813 -1.10168153  1.89169776 -2.05578153
 -0.65473562  0.43983786 -2.44876706 -3.17748648  1.70635113 -0.00889219
 -1.2337575  -0.28339179 -0.58022525 -1.73020074 -2.0530971  -0.31450634
  0.0889827  -1.13289081  2.41530057 -1.78109017  1.6141715  -2.89051295
 -0.325372    0.85604295  1.78428422 -3.89878151  1.59904976  0.87987374
 -0.33651884  0.40605941 -0.99069674  0.19412857  0.52400221  1.98860042
 -0.23403622  1.20314338  0.37623285  1.93702102  0.41455776  2.21730615
  1.66991407  0.36029161 

eval_count: 101
var_params: [-0.40863047  2.81048065  1.98098866 -1.07398453  0.47539432 -0.17386323
  2.21953975 -0.28944472  1.15397632  1.25196228  2.21913385 -2.47137159
  0.33529074  0.89242869 -1.76829786  0.13441615  1.18598171 -0.33243418
 -0.56399872  1.97198384 -1.59461414 -0.51738264 -0.28702485  0.8061582
 -0.72097161 -0.51889939 -1.47855225 -1.4734869   0.30988388 -0.03363449
 -0.29330159  0.67444849  1.46786039 -0.19356308  0.33150514 -0.22177175
 -0.23100385 -1.87435223  1.36506707 -1.0938985   1.88391472 -2.18555702
 -0.53039816  0.45294853 -2.59943624 -3.30182395  1.69313007 -0.02211325
 -1.10942004 -0.13272261 -0.71000074 -1.73787337 -2.19832826 -0.327617
  0.23421386 -1.27812197  2.54507606 -1.7888732   1.46350231 -2.903734
 -0.47604118  0.84837032  1.77661158 -3.90645415  1.61227081  0.87220111
 -0.34430187  0.27628392 -0.98291371  0.18645594  0.51078115  1.83793124
 -0.08880506  1.32748085  0.24645736  2.06135849  0.40677473  2.36797534
  1.6568034   0.50552277 -0.

eval_count: 107
var_params: [-2.68169976e-01  3.12585969e+00  2.20507373e+00 -1.29806959e+00
  5.63008102e-01  5.02218283e-02  2.39844751e+00 -5.76902491e-02
  1.37806138e+00  9.36583243e-01  2.30674763e+00 -2.38375781e+00
  6.50669779e-01  7.13520933e-01 -1.85591164e+00 -8.96689166e-02
  1.27359549e+00 -5.11341936e-01 -7.04459213e-01  2.05959762e+00
 -1.36285967e+00 -4.29768859e-01 -5.52703826e-02  4.90779159e-01
 -7.24960816e-01 -3.39991635e-01 -1.65746000e+00 -1.61394739e+00
  5.33968939e-01 -2.57719557e-01 -5.25056057e-01  8.98533553e-01
  1.23610592e+00 -2.81176856e-01  2.36221954e-01 -8.13112523e-02
 -2.27014643e-01 -1.55897319e+00  1.36107786e+00 -8.62144026e-01
  1.65216025e+00 -2.36446478e+00 -3.06313095e-01  6.77033588e-01
 -2.45897575e+00 -3.61720299e+00  1.91721514e+00  1.56794511e-01
 -9.30512281e-01 -4.51088258e-02 -5.69540242e-01 -1.91678113e+00
 -2.05786776e+00 -1.87156510e-01  4.13121621e-01 -1.45702973e+00
  2.72398382e+00 -2.02062768e+00  1.14812327e+00 -3.13548848e+

eval_count: 112
var_params: [-8.82829077e-02  3.30574676e+00  2.04258960e+00 -1.27115063e+00
  3.83121034e-01  2.33028705e-02  2.56093163e+00 -8.05443485e-02
  1.56201331e+00  7.56696175e-01  2.12279571e+00 -2.22127369e+00
  6.23750821e-01  5.51036809e-01 -1.67602457e+00 -6.27499588e-02
  1.30051445e+00 -6.73826060e-01 -8.88411140e-01  2.22208174e+00
 -1.52534379e+00 -2.67284734e-01  1.28681544e-01  5.17698117e-01
 -7.51879774e-01 -1.77507511e-01 -1.68031410e+00 -1.59109329e+00
  3.75549673e-01 -9.52354329e-02 -7.09007984e-01  8.75679453e-01
  1.05215400e+00 -3.04030955e-01  2.09302996e-01 -1.04165352e-01
 -2.00095686e-01 -1.51465128e+00  1.40946463e+00 -1.02462815e+00
  1.60377349e+00 -2.18457771e+00 -2.61991193e-01  5.18614322e-01
 -2.41465385e+00 -3.45878373e+00  2.09710221e+00  1.12472610e-01
 -9.03593323e-01 -2.22547264e-02 -5.96459200e-01 -2.07520040e+00
 -1.87391584e+00 -2.14075468e-01  5.71540887e-01 -1.27307780e+00
  2.88240308e+00 -2.20051475e+00  1.09973651e+00 -2.97706921e+

eval_count: 118
var_params: [ 1.52934156e-01  3.24600894e+00  2.22696647e+00 -1.22149437e+00
  5.67497903e-01  2.64519934e-01  2.79206714e+00  2.59521096e-02
  1.36755488e+00  6.50199717e-01  2.31725413e+00 -2.12485879e+00
  4.39373951e-01  3.19901304e-01 -1.86040144e+00 -3.03967022e-01
  1.05929738e+00 -9.15043124e-01 -8.38754875e-01  2.32857820e+00
 -1.33088537e+00 -4.51661604e-01  2.35178002e-01  2.86562612e-01
 -8.11617596e-01 -8.10926114e-02 -1.74005193e+00 -1.82222880e+00
  2.79134774e-01  1.12610252e-02 -6.49270161e-01  1.10681496e+00
  9.55739097e-01 -2.07616056e-01  1.48445679e-02  2.33110646e-03
 -1.57188161e-02 -1.75586835e+00  1.30296817e+00 -8.30169722e-01
  1.70018839e+00 -1.94336064e+00 -5.03208256e-01  4.22199423e-01
 -2.35491602e+00 -3.22764822e+00  1.91272534e+00  5.27347866e-02
 -7.19216454e-01 -1.28751185e-01 -4.89962742e-01 -1.89082353e+00
 -1.63269877e+00 -1.07579009e-01  8.12757951e-01 -1.46753623e+00
  2.68794465e+00 -2.44173181e+00  9.15359642e-01 -2.73585215e+

In [ ]:
print(result)

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [217]:
print(repr(vqc.optimal_params))

array([ 0.54632732,  0.40538044, -1.01295329,  1.49487678,  1.64501921,
        0.59286665,  2.12784723,  1.21054489, -0.5643853 ,  0.95213542,
       -0.27556139, -1.83490042, -0.01200562,  3.75627203,  1.43557252,
       -0.39997671, -0.5545942 ,  0.46746988, -1.27218015,  0.0318519 ,
        0.87543005, -0.25437162, -1.34021757, -2.08737164,  2.04253738,
        0.83175021,  0.77956833, -0.17427723,  2.16326879, -1.5828532 ,
        1.17114289, -1.17920076, -1.45410869, -0.89369175,  0.01718402,
       -1.59692096,  0.34737521, -2.48543596,  0.74288619,  0.28864511,
        0.57559697,  0.51204554, -0.92355601,  0.49001691, -0.6171768 ,
        0.14289195,  1.87852521,  0.44203041,  2.02773329,  1.01900935,
        1.40938457, -0.3939512 ,  0.50795543,  0.43727053,  0.64402482,
        0.43716062, -2.43465227,  0.81417097,  0.38247537, -0.64960273,
       -0.71656997,  0.64017647,  0.91821865, -0.04276097, -2.2802225 ,
       -0.90311979,  1.9641994 , -0.9093311 , -0.08099036,  0.13

In [218]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [ 0.54632732,  0.40538044, -1.01295329,  1.49487678,  1.64501921,
        0.59286665,  2.12784723,  1.21054489, -0.5643853 ,  0.95213542,
       -0.27556139, -1.83490042, -0.01200562,  3.75627203,  1.43557252,
       -0.39997671, -0.5545942 ,  0.46746988, -1.27218015,  0.0318519 ,
        0.87543005, -0.25437162, -1.34021757, -2.08737164,  2.04253738,
        0.83175021,  0.77956833, -0.17427723,  2.16326879, -1.5828532 ,
        1.17114289, -1.17920076, -1.45410869, -0.89369175,  0.01718402,
       -1.59692096,  0.34737521, -2.48543596,  0.74288619,  0.28864511,
        0.57559697,  0.51204554, -0.92355601,  0.49001691, -0.6171768 ,
        0.14289195,  1.87852521,  0.44203041,  2.02773329,  1.01900935,
        1.40938457, -0.3939512 ,  0.50795543,  0.43727053,  0.64402482,
        0.43716062, -2.43465227,  0.81417097,  0.38247537, -0.64960273,
       -0.71656997,  0.64017647,  0.91821865, -0.04276097, -2.2802225 ,
       -0.90311979,  1.9641994 , -0.9093311 , -0.08099036,  0.13153073,
        0.06269059,  0.90468536]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [219]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [220]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [221]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [222]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [223]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 169.226498146003 seconds
Accuracy of the model is 0.81 and circuit cost is 876


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [52]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.6835
Circuit Cost: 648
The complete unrolled circuit: 


┌───────────────┐                                                 »
q_0: ┤ U3(pi/2,0,pi) ├──────────────────────■───────────────────────■──»
     ├───────────────┤┌──────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐»
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,-0.99523) ├┤ X ├┤ U3(0,0,0.99523) ├┤ X ├»
     ├───────────────┤└┬───────────────┬─┘└───┘└─────────────────┘└───┘»
q_2: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,2.597) ├───────────────────────────────»
     └───────────────┘ └───────────────┘                               »
c_0: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_1: ══════════════════════════════════════════════════════════════════»
                                                                       »
c_2: ══════════════════════════════════════════════════════════════════»
                                                                       »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X ├»
«     └─────────────────┘                       ┌───────────────────────┐└─┬─┘»
«q_1: ─────────■─────────────────────────────■──┤ U3(0.8228,-pi/2,pi/2) ├──┼──»
«            ┌─┴─┐       ┌────────────────┐┌─┴─┐└───────────────────────┘  │  »
«q_2: ───────┤ X ├───────┤ U3(0,0,-2.597) ├┤ X ├───────────────────────────■──»
«            └───┘       └────────────────┘└───┘                              »
«c_0: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_1: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«c_2: ════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌────────────────┐                    ┌───┐   ┌──────────────────┐   »
«q_0: ┤ U3(0,0,0.9614) ├────────────────────┤ X ├───┤ U3(4,-pi/2,pi/2) ├───»
«     ├───────────────┬┘┌──────────────────┐└─┬─┘   └──────────────────┘   »
«q_1: ┤ U3(pi/2,0,pi) ├─┤ U3(0,0,-0.99523) ├──┼────────────────────────────»
«     └───────────────┘ └──────────────────┘  │  ┌────────────────────────┐»
«q_2: ────────────────────────────────────────■──┤ U3(0.90163,-pi/2,pi/2) ├»
«                                                └────────────────────────┘»
«c_0: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_1: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_2: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────────────┐                                         »
«q_0: ┤ U3(pi/2,0,pi) ├────────■─────────────────────────────■──»
«     └───────────────┘      ┌─┴─┐      ┌─────────────────┐┌─┴─┐»
«q_1: ───────────────────────┤ X ├──────┤ U3(0,0,0.99523) ├┤ X ├»
«     ┌───────────────┐┌─────┴───┴─────┐└─────────────────┘└───┘»
«q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,2.597) ├────────────────────────»
«     └───────────────┘└───────────────┘                        »
«c_0: ══════════════════════════════════════════════════════════»
«                                                               »
«c_1: ══════════════════════════════════════════════════════════»
«                                                               »
«c_2: ══════════════════════════════════════════════════════════»
«                                                               »
«     ┌─────────────────┐                                                ┌───┐»
«q_0: ┤ U3(0,0,-0.9614) ├────────────────────────────────────────────────┤ X 